In [1]:
import numpy as np
import pandas as pd

In [2]:
data_path = "D:\\University\\DataScienceWorkshop\\"

In [7]:
train = pd.read_csv(data_path + "train_sample.csv", dtype={"clicked" : bool})
events = pd.read_csv(data_path + "events_sample.csv", usecols=[0,2])

In [9]:
#add number of ads on display
train_ad_count_per_display = train.groupby(['display_id'])['display_id'].agg({'ads_on_doc' : 'count'}).reset_index()
train = train.merge(train_ad_count_per_display, how = 'left', on = 'display_id')
del train_ad_count_per_display

In [11]:
#add documents to train table
train = train.merge(events, how = 'left', on = 'display_id')

In [18]:
train.dropna(inplace=True)

In [21]:
train.document_id = train.document_id.astype(int)

In [30]:
train["ad_probability"] = 1 / train["ads_on_doc"]

In [24]:
train = train.groupby(['document_id','ad_id','ads_on_doc']).clicked.agg({'clicked' : 'sum', 'total' : 'count'}).reset_index()

In [26]:
#preparing columns for computing ad document strength
train['clicked_percent'] = train['clicked'] / train['total']
train['clicked_percent_normalized'] = (train['clicked'] + 12 * train['clicked_percent'].mean()) / (12 + train['total'])
train['likelihood_normalized'] = train['clicked_percent_normalized'] / train['ad_probability']
train['sum_likelihood'] = train['likelihood_normalized'] * train['total']

In [57]:
#computing strength from selected features
ad_doc_strength = train.drop(train.columns[[2, 3]],axis = 1).groupby(['document_id','ad_id']).sum().reset_index()
ad_doc_strength['strength'] = ad_doc_strength['sum_likelihood'] / ad_doc_strength['total']
ad_doc_strength.drop(ad_doc_strength.columns[[2,5]],axis = 1, inplace=True)
ad_doc_strength.sort_values(by='strength', ascending=False)

,document_id,ad_id,clicked_percent,clicked_percent_normalized,likelihood_normalized,sum_likelihood,strength
117817,961361,132019,0.952381,0.674306,5.394451,113.283462,5.394451
117661,961313,234107,1.000000,0.625081,5.000649,70.009084,5.000649
127124,1026091,258183,0.640000,0.666539,6.492155,124.884211,4.803239
257560,1675109,165380,1.000000,0.593838,4.750703,57.008435,4.750703
126903,1026091,39068,0.600000,0.445378,4.453784,89.075679,4.453784
324418,1763695,111199,1.722222,0.758566,6.576930,84.362685,4.440141
257366,1675092,280430,0.812500,0.544718,4.357745,69.723925,4.357745
265038,1690149,184611,2.800212,2.005203,13.584685,1210.303361,4.322512
133559,1060089,257136,0.636364,0.478003,4.302029,94.644633,4.302029
44019,354745,289670,1.000000,0.426595,4.265946,21.329731,4.265946


In [53]:
ad_doc_strength

,document_id,ad_id,ads_on_doc,clicked_percent,clicked_percent_normalized,ad_probability,likelihood_normalized,sum_likelihood
0,12,195273,4,0.5,0.232293,0.250000,0.929174,1.858348
1,12,213466,4,0.0,0.173239,0.250000,0.692957,0.692957
2,12,221437,4,0.0,0.173239,0.250000,0.692957,0.692957
3,12,241409,4,0.0,0.173239,0.250000,0.692957,0.692957
4,12,252531,4,0.0,0.173239,0.250000,0.692957,0.692957
5,12,257743,4,1.0,0.250162,0.250000,1.000649,1.000649
6,12,288356,4,0.0,0.173239,0.250000,0.692957,0.692957
7,12,335714,4,0.0,0.160865,0.250000,0.643460,1.286919
8,12,353895,4,0.0,0.173239,0.250000,0.692957,0.692957
9,12,371497,4,1.0,0.250162,0.250000,1.000649,1.000649


In [55]:
train

,document_id,ad_id,ads_on_doc,clicked,total,clicked_percent,clicked_percent_normalized,ad_probability,likelihood_normalized,sum_likelihood
0,12,195273,4,1.0,2,0.5,0.232293,0.250000,0.929174,1.858348
1,12,213466,4,0.0,1,0.0,0.173239,0.250000,0.692957,0.692957
2,12,221437,4,0.0,1,0.0,0.173239,0.250000,0.692957,0.692957
3,12,241409,4,0.0,1,0.0,0.173239,0.250000,0.692957,0.692957
4,12,252531,4,0.0,1,0.0,0.173239,0.250000,0.692957,0.692957
5,12,257743,4,1.0,1,1.0,0.250162,0.250000,1.000649,1.000649
6,12,288356,4,0.0,1,0.0,0.173239,0.250000,0.692957,0.692957
7,12,335714,4,0.0,2,0.0,0.160865,0.250000,0.643460,1.286919
8,12,353895,4,0.0,1,0.0,0.173239,0.250000,0.692957,0.692957
9,12,371497,4,1.0,1,1.0,0.250162,0.250000,1.000649,1.000649
